In [11]:
import pandas as pd
import re

In [2]:
url = 'https://wiki.openstreetmap.org/wiki/OSM_tags_for_routing/Access_restrictions'

dfs = pd.read_html(url)

In [3]:
dfs_access = dict()
dfs_codes=['Default','AUS','AUT','BLR','BEL','BRA','DNK','FRA','FIN','DEU','CHN','HUN','ISL','ITA','NLD','NOR','OMN','PHL','POL','ROU','RUS','SVK','ESP','CHE','TUR','UKR','GBR','USA']
for i in range(0, len(dfs)):
    dfs_access[dfs_codes[i]]=dfs[i]

In [4]:
#in each cell remove parenthesis, strip if empty: no, same with *
#change tbd,  , dismount, destination with yes partial|designated|yes|no
# put everything lowercase
import re

regex = r"(\([^)]+\))|(\[[^)]+\])|[^a-zA-Z]"

def clean_df(item):
    item = item.strip().lower()
    item = re.sub(regex, ' ', item, 0).strip()
    if item == 'tbd' or len(item)==0:
        item = 'no'
    return item

header_items = ['highway','access','motorcar','motorcycle','goods','hgv','psv','moped','horse','bicycle','foot' ]

In [5]:
dfs_access['Default']

,highway=*,access,motorcar,motorcycle,goods,hgv,psv,moped,horse,bicycle,foot
0,motorway motorway_link,﻿yes,Designated,Designated,Designated,Designated,Designated,﻿no,﻿no,﻿no,﻿no
1,trunk trunk_link,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes
2,primary primary_link secondary secondary_link ...,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes,﻿yes
3,pedestrian,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿yes
4,path,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿yes,﻿yes,﻿yes
5,bridleway,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,Designated,﻿no,﻿no
6,cycleway,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,Designated,﻿no
7,footway,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,﻿no,Designated


In [6]:
new_df = dfs_access['BRA'].applymap(clean_df)
dfs = dict()
for key,df in dfs_access.items():
    new_df = df.applymap(clean_df)
    for header_item in header_items:
        for column in new_df.columns:
            if header_item in column.lower():
                new_df[header_item] = new_df[column]
                break
        
    delete_these = set(new_df.columns) - set(header_items)
    new_df = new_df.drop(columns=delete_these)
    dfs[key.lower()] = new_df
dfs['fra'].head(n=2)


,motorcar,motorcycle,goods,hgv,psv,moped,horse,bicycle,foot,highway
0,yes,yes,yes,yes,yes,no,no,no,no,motorway trunk
1,yes,yes,yes,yes,yes,yes,yes,yes,yes,primary secondary tertiary unclassified reside...


In [40]:
levels = {'levels': [
    {0: ['construction', 'demolished', 'raceway', 'abandoned', 'disused', 'foo', 'no','projected', 'planned','proposed','razed','dismantled','historic']},
    {1: ['stairway', 'stairs',  'elevator', 'corridor', 'hallway', 'slide']},
    {2: ['services', 'bus', 'busway', 'bus_guideway', 'access', 'bus_stop', 'via_ferrata', 'access_ramp', 'emergency_access_point', 'emergency_bay','service', 'footway',
         'traffic_island', 'virtual', 'cyleway', 'cycleway', 'byway', 'path', 'track', 'pedestrian', 'steps',
         'platform', 'bridleway', 'rest_area', 'escape']
     },
    {3: ['residential', 'yes', 'unclassified', 'crossing', 'unknown',
         'bridge', 'lane', 'ford', 'psv', 'living_street','alley']
     },
    {4: ['tertiary', 'tertiary_link', 'turning_circle', 'road', 'roundabout', 'ice_road']},
    {5: ['secondary', 'secondary_link']},
    {6: ['primary', 'primary_link']},
    {7: ['trunk', 'trunk_link']},
    {8: ['motorway', 'motorway_link','ramp']}
], 'public_transport': [
    {0: 'Buses'},  # PSV lanes, busway, bus guideway, routes, stops, platform, stations
    {1: 'Trolleybuses'},  # Routes, wires, stops
    {2: 'Trains'},  # Tracks, railway routes, train routes, halt, stations
    {3: 'Light rail'},  # Tracks, routes, stations
    {4: 'Metros'},  # Tracks, routes, stations, entrances
    {5: 'Trams'},  # Tracks, routes, stops
    {6: 'Aerialways'},  # cable car, gondola, chair lift, drag lift, zip line, stations, pylons
    {7: 'Ferries'},  # Shipping lanes, routes, ports, docks
    {8: 'Flights'}  # Runways, taxiways, aprons, terminals, aerodromes, hangars, heliports, helipads
]
}

levels_2 = {}
for d in levels['levels']:
    for l, hs in d.items():
        for h in hs:
            levels_2[h] = l
print(levels_2)

{'construction': 0, 'demolished': 0, 'raceway': 0, 'abandoned': 0, 'disused': 0, 'foo': 0, 'no': 0, 'projected': 0, 'planned': 0, 'proposed': 0, 'razed': 0, 'dismantled': 0, 'historic': 0, 'stairway': 1, 'stairs': 1, 'elevator': 1, 'corridor': 1, 'hallway': 1, 'slide': 1, 'services': 2, 'bus': 2, 'busway': 2, 'bus_guideway': 2, 'access': 2, 'bus_stop': 2, 'via_ferrata': 2, 'access_ramp': 2, 'emergency_access_point': 2, 'emergency_bay': 2, 'service': 2, 'footway': 2, 'traffic_island': 2, 'virtual': 2, 'cyleway': 2, 'cycleway': 2, 'byway': 2, 'path': 2, 'track': 2, 'pedestrian': 2, 'steps': 2, 'platform': 2, 'bridleway': 2, 'rest_area': 2, 'escape': 2, 'residential': 3, 'yes': 3, 'unclassified': 3, 'crossing': 3, 'unknown': 3, 'bridge': 3, 'lane': 3, 'ford': 3, 'psv': 3, 'living_street': 3, 'alley': 3, 'tertiary': 4, 'tertiary_link': 4, 'turning_circle': 4, 'road': 4, 'roundabout': 4, 'ice_road': 4, 'secondary': 5, 'secondary_link': 5, 'primary': 6, 'primary_link': 6, 'trunk': 7, 'trunk_

In [51]:
regex = r"(?P<link>\w+)\slink"
subst = "\\g<link>_link"

access_dict  = {}
for country in dfs.keys():
    access_dict[country] = {}
    access_highway = []
    access_level = []
    for item in dfs[country].itertuples(index=False):
        highway = item.highway
        highway = re.sub(regex, subst, highway, 0, re.MULTILINE)
        highway = highway.replace('living street', 'living_street')
        highway = highway.replace('footway sidewalk', 'sidewalk')
        highway = highway.replace('footway crossing', 'crossing')
        item_dict = item._asdict()
        
        
        
        for h in list(set(highway.split())):
            if len(h) < 1 or h in [a['highway'] for a in access_highway]:
                continue
            
            i_d = item_dict.copy()
            i_d['highway'] = h            
            access_highway.append(i_d)
            
            h_level = levels_2[h] if h in levels_2.keys() else 3
            i_d = item_dict.copy()
            i_d['level'] = h_level
            del i_d['highway']
            if h_level not in [a['level'] for a in access_level]:
                access_level.append(i_d)

    access_dict[country]['highway'] = pd.DataFrame.from_records(access_highway, index='highway').to_dict('index')
    access_dict[country]['level'] = pd.DataFrame.from_records(access_level, index='level').to_dict('index')
access_dict['default']['level']

{8: {'access': 'yes',
  'motorcar': 'designated',
  'motorcycle': 'designated',
  'goods': 'designated',
  'hgv': 'designated',
  'psv': 'designated',
  'moped': 'no',
  'horse': 'no',
  'bicycle': 'no',
  'foot': 'no'},
 7: {'access': 'yes',
  'motorcar': 'yes',
  'motorcycle': 'yes',
  'goods': 'yes',
  'hgv': 'yes',
  'psv': 'yes',
  'moped': 'yes',
  'horse': 'yes',
  'bicycle': 'yes',
  'foot': 'yes'},
 3: {'access': 'yes',
  'motorcar': 'yes',
  'motorcycle': 'yes',
  'goods': 'yes',
  'hgv': 'yes',
  'psv': 'yes',
  'moped': 'yes',
  'horse': 'yes',
  'bicycle': 'yes',
  'foot': 'yes'},
 5: {'access': 'yes',
  'motorcar': 'yes',
  'motorcycle': 'yes',
  'goods': 'yes',
  'hgv': 'yes',
  'psv': 'yes',
  'moped': 'yes',
  'horse': 'yes',
  'bicycle': 'yes',
  'foot': 'yes'},
 6: {'access': 'yes',
  'motorcar': 'yes',
  'motorcycle': 'yes',
  'goods': 'yes',
  'hgv': 'yes',
  'psv': 'yes',
  'moped': 'yes',
  'horse': 'yes',
  'bicycle': 'yes',
  'foot': 'yes'},
 4: {'access': 'yes

In [52]:
access_dict['default']['level'][1] = {'access': 'no',
  'motorcar': 'no',
  'motorcycle': 'no',
  'goods': 'no',
  'hgv': 'no',
  'psv': 'no',
  'moped': 'no',
  'horse': 'no',
  'bicycle': 'no',
  'foot': 'yes'}
access_dict['default']['level'][0] = {'access': 'no',
  'motorcar': 'no',
  'motorcycle': 'no',
  'goods': 'no',
  'hgv': 'no',
  'psv': 'no',
  'moped': 'no',
  'horse': 'no',
  'bicycle': 'no',
  'foot': 'no'}

In [54]:
print(access_dict)

{'default': {'highway': {'motorway_link': {'access': 'yes', 'motorcar': 'designated', 'motorcycle': 'designated', 'goods': 'designated', 'hgv': 'designated', 'psv': 'designated', 'moped': 'no', 'horse': 'no', 'bicycle': 'no', 'foot': 'no'}, 'motorway': {'access': 'yes', 'motorcar': 'designated', 'motorcycle': 'designated', 'goods': 'designated', 'hgv': 'designated', 'psv': 'designated', 'moped': 'no', 'horse': 'no', 'bicycle': 'no', 'foot': 'no'}, 'trunk': {'access': 'yes', 'motorcar': 'yes', 'motorcycle': 'yes', 'goods': 'yes', 'hgv': 'yes', 'psv': 'yes', 'moped': 'yes', 'horse': 'yes', 'bicycle': 'yes', 'foot': 'yes'}, 'trunk_link': {'access': 'yes', 'motorcar': 'yes', 'motorcycle': 'yes', 'goods': 'yes', 'hgv': 'yes', 'psv': 'yes', 'moped': 'yes', 'horse': 'yes', 'bicycle': 'yes', 'foot': 'yes'}, 'unclassified': {'access': 'yes', 'motorcar': 'yes', 'motorcycle': 'yes', 'goods': 'yes', 'hgv': 'yes', 'psv': 'yes', 'moped': 'yes', 'horse': 'yes', 'bicycle': 'yes', 'foot': 'yes'}, 'seco

In [93]:
import pickle
with open('access-dataframe.pkl','wb') as fp:
    pickle.dump(dfs,fp)

In [107]:
dfs['fra'].loc[dfs['fra']['highway'].str.contains('bique')].get('bicycle').max()

nan

In [89]:
import numpy as np

def search_highway_access(country_iso3, highway, access_type):
    if country_iso3 in dfs.keys(): 
        access = dfs[country_iso3].loc[dfs[country_iso3]['highway'].str.contains(highway)].get(access_type).max()
        print(access, type(access))
        if isinstance(access,str):
            if access not in ['no']:
                return True
    access = dfs['default'].loc[dfs['default']['highway'].str.contains(highway)].get(access_type).max()
    if isinstance(access,str):
        if access not in ['no']:
            return True
    return False

In [91]:
print(search_highway_access('fra','tertiary','bicycle'))
print(search_highway_access('fra','cycleway','bicycle'))

yes <class 'str'>
True
designated <class 'str'>
True


In [99]:
with open('../../policosm/utils/countries-epsg.pickle','rb') as fp:
    a = pickle.load(fp)
pickle.dumps(a)

b'\x80\x03cgeopandas.geodataframe\nGeoDataFrame\nq\x00)\x81q\x01}q\x02(X\x05\x00\x00\x00_dataq\x03cpandas.core.internals.managers\nBlockManager\nq\x04)\x81q\x05(]q\x06(cpandas.core.indexes.base\n_new_Index\nq\x07cpandas.core.indexes.base\nIndex\nq\x08}q\t(X\x04\x00\x00\x00dataq\ncnumpy.core.multiarray\n_reconstruct\nq\x0bcnumpy\nndarray\nq\x0cK\x00\x85q\rC\x01bq\x0e\x87q\x0fRq\x10(K\x01K\x03\x85q\x11cnumpy\ndtype\nq\x12X\x02\x00\x00\x00O8q\x13K\x00K\x01\x87q\x14Rq\x15(K\x03X\x01\x00\x00\x00|q\x16NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?tq\x17b\x89]q\x18(X\x04\x00\x00\x00nameq\x19X\x04\x00\x00\x00epsgq\x1aX\x08\x00\x00\x00geometryq\x1betq\x1cbX\x04\x00\x00\x00nameq\x1dNu\x86q\x1eRq\x1fh\x07cpandas.core.indexes.range\nRangeIndex\nq }q!(h\x1dNX\x05\x00\x00\x00startq"K\x00X\x04\x00\x00\x00stopq#K\xa9X\x04\x00\x00\x00stepq$K\x01u\x86q%Rq&e]q\'(h\x0bh\x0cK\x00\x85q(h\x0e\x87q)Rq*(K\x01K\x02K\xa9\x86q+h\x15\x89]q,(X\x07\x00\x00\x00Montanaq-X\r\x00\x00\x00West Virginiaq.h.X\x0c\x00\x00\x00Pennsy